# Domain Modeling by C#

In [50]:
public interface IReference {
    public String Reference { get; init; }
}

public abstract class ReferenceIdentity<T> : IEquatable<T> where T: IReference {
    public abstract String Reference { get; init; }

    public bool Equals(T other) { return this.Reference == other.Reference; }
    public override bool Equals(object o) { return this.Equals(o as IEquatable<T>); }
    public override int GetHashCode() { return this.Reference.GetHashCode(); }

    public static bool operator ==(ReferenceIdentity<T> lhs, ReferenceIdentity<T> rhs) {
        return lhs.Reference == rhs.Reference;
    }
    
    public static bool operator !=(ReferenceIdentity<T> lhs, ReferenceIdentity<T> rhs)
    {
        return lhs.Reference != rhs.Reference;
    }           
}

In [54]:
public class Batch : ReferenceIdentity<Batch>, IReference {
    public override String Reference { get; init; }
    public String Sku { get; init; }
    public int Qty { get; init; }
    public DateTime Eta { get; set; } = DateTime.Today;
    public List<OrderLine> Allocations { get; init; } = new();
};

public record OrderLine {
    public String OrderID { get; init; }
    public String Sku { get; init; }
    public int Qty { get; init; }
};

public class Order : ReferenceIdentity<Order>, IReference {
    public override String Reference { get; init; }
    public String ID { get; init; }
    public List<OrderLine> OrderLines { get; init; }
}

In [55]:
var batch1 = new Batch { Reference="batch-000", Sku="TEST-TABLE", Qty=10 };
var batch2 = new Batch { Reference="batch-001", Sku="TEST-TABLE", Qty=10 };
var batch3 = new Batch { Reference="batch-000", Sku="TEST-CHAIR", Qty=3 };

In [57]:
batch1 == batch2

False

In [59]:
batch1 == batch3

True

### Unit testing by xUnit 

- https://docs.microsoft.com/en-us/dotnet/core/testing/unit-testing-with-dotnet-test


```
$ dotnet new sln -o csharp
$ cd csharp
$ dotnet new xunit -o Models.Tests
$ dotnet sln add ./Models.Tests/Models.Tests.csproj
```

In [151]:
#r "nuget:xunit,2.4.1"

Installed package xunit.runner.console version 2.4.1

In [89]:
using System;
using Xunit;

public class UnitTest1 {
    [Fact]
    public void Test1()
    {
        var batch1 = new Batch { Reference="batch-000", Sku="TEST-TABLE", Qty=10 };
        var batch2 = new Batch { Reference="batch-001", Sku="TEST-TABLE", Qty=10 };
        var batch3 = new Batch { Reference="batch-000", Sku="TEST-CHAIR", Qty=3 };
        Assert.Equal(batch1, batch2);
    }
}

In [154]:
// Jupyter Notebook에서 테스트 돌리기 위해 Reflection으로 만든 간이 Test Runner
public void runTests(Type type) {
    var instance = Activator.CreateInstance(type);
    
    var methods = (
        from it in type.GetMethods() 
        where it.CustomAttributes.Count() > 0 && 
        it.CustomAttributes.ElementAt(0).AttributeType.FullName.StartsWith("Xunit.")
        select it
    );
        
    foreach(var method in methods) {
        Console.WriteLine(method.Name);
        method.Invoke(instance, null);
    }
}

In [155]:
runTests(typeof(UnitTest1))

Test1


Unhandled exception: System.Reflection.TargetInvocationException: Exception has been thrown by the target of an invocation.
 ---> Xunit.Sdk.EqualException: Assert.Equal() Failure
Expected: Batch { Allocations = [], Eta = 2020-12-16T00:00:00.0000000+09:00, Qty = 10, Reference = "batch-000", Sku = "TEST-TABLE" }
Actual:   Batch { Allocations = [], Eta = 2020-12-16T00:00:00.0000000+09:00, Qty = 10, Reference = "batch-001", Sku = "TEST-TABLE" }
   at Xunit.Assert.Equal[T](T expected, T actual, IEqualityComparer`1 comparer) in C:\Dev\xunit\xunit\src\xunit.assert\Asserts\EqualityAsserts.cs:line 40
   at Xunit.Assert.Equal[T](T expected, T actual) in C:\Dev\xunit\xunit\src\xunit.assert\Asserts\EqualityAsserts.cs:line 24
   at Submission#90.UnitTest1.Test1()
   --- End of inner exception stack trace ---
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Object[] arguments, Signature sig, Boolean constructor, Boolean wrapExceptions)
   at Submission#154.runTests(Type type)
   at Submi